# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\16462\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\16462\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql("SELECT * FROM Messages", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # normalization word tokens and remove stop words
    normlizer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    normlized = [normlizer.stem(word) for word in tokens if word not in stop_words]
    
    return normlized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline1 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size = 0.2, random_state = 45)

pipeline1.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000206A27C23A0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def plot_scores(Y_test, Y_pred):
    #Testing the model
    # Printing the classification report for each label
    i = 0
    for col in Y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(Y_test[col], Y_pred[:, i]))
        i = i + 1
    accuracy = (Y_pred == Y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))

In [7]:
# Prediction: the Random Forest Classifier  
Y_pred = pipeline1.predict(X_test)
plot_scores(Y_test, Y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.70      0.44      0.54      1232
           1       0.84      0.94      0.89      4008
           2       0.35      0.39      0.37        38

    accuracy                           0.82      5278
   macro avg       0.63      0.59      0.60      5278
weighted avg       0.81      0.82      0.80      5278

Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      4399
           1       0.82      0.50      0.62       879

    accuracy                           0.90      5278
   macro avg       0.86      0.74      0.78      5278
weighted avg       0.89      0.90      0.89      5278

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5253
           1       1.00      0.04      0.08        25

    accuracy                           1.00      5278
   macro avg       

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
# Show parameters for the pipline
pipeline1.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000206A27C23A0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000206A27C23A0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [9]:
# Using grid search
# Create Grid search parameters for Random Forest Classifier   
parameters = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 20]
}

cv = GridSearchCV(pipeline1, param_grid = parameters)
cv

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000206A27C23A0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 20],
                         'tfidf__use_idf': (True, False)})

In [10]:
cv


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000206A27C23A0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 20],
                         'tfidf__use_idf': (True, False)})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
# Fit the first tuned model
cv.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000206A27C23A0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 20],
                         'tfidf__use_idf': (True, False)})

In [12]:
# Predicting using the first tuned model 
Y_pred = cv.predict(X_test)
plot_scores(Y_test, Y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.67      0.46      0.55      1232
           1       0.85      0.92      0.89      4008
           2       0.34      0.53      0.41        38

    accuracy                           0.81      5278
   macro avg       0.62      0.64      0.62      5278
weighted avg       0.80      0.81      0.80      5278

Feature 2: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4399
           1       0.80      0.46      0.58       879

    accuracy                           0.89      5278
   macro avg       0.85      0.72      0.76      5278
weighted avg       0.88      0.89      0.88      5278

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5253
           1       1.00      0.04      0.08        25

    accuracy                           1.00      5278
   macro avg       

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5224
           1       0.00      0.00      0.00        54

    accuracy                           0.99      5278
   macro avg       0.49      0.50      0.50      5278
weighted avg       0.98      0.99      0.98      5278

Feature 28: other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5074
           1       0.00      0.00      0.00       204

    accuracy                           0.96      5278
   macro avg       0.48      0.50      0.49      5278
weighted avg       0.92      0.96      0.94      5278

Feature 29: weather_related
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      3824
           1       0.85      0.65      0.74      1454

    accuracy                           0.87      5278
   macro avg       0.86      0.80      0.83      5278
weighted avg 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
# using Adaboost Classifier instead 
from sklearn.ensemble import AdaBoostClassifier
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
])

parameters2 =  {'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 60, 70]} 
cv2 = GridSearchCV(pipeline2, param_grid = parameters2)



In [14]:
cv2

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000206A27C23A0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 60, 70],
                         'tfidf__use_idf': (True, False)})

In [15]:
cv2.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000206A27C23A0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 60, 70],
                         'tfidf__use_idf': (True, False)})

In [16]:
# Predicting using the first tuned model 
Y_pred = cv2.predict(X_test)
plot_scores(Y_test, Y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.71      0.10      0.17      1232
           1       0.78      0.99      0.87      4008
           2       0.50      0.18      0.27        38

    accuracy                           0.77      5278
   macro avg       0.66      0.42      0.44      5278
weighted avg       0.76      0.77      0.70      5278

Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.96      0.93      4399
           1       0.73      0.50      0.59       879

    accuracy                           0.89      5278
   macro avg       0.82      0.73      0.76      5278
weighted avg       0.88      0.89      0.88      5278

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5253
           1       0.22      0.08      0.12        25

    accuracy                           0.99      5278
   macro avg       

Feature 27: aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5224
           1       0.20      0.06      0.09        54

    accuracy                           0.99      5278
   macro avg       0.60      0.53      0.54      5278
weighted avg       0.98      0.99      0.98      5278

Feature 28: other_infrastructure
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      5074
           1       0.41      0.13      0.20       204

    accuracy                           0.96      5278
   macro avg       0.69      0.56      0.59      5278
weighted avg       0.94      0.96      0.95      5278

Feature 29: weather_related
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      3824
           1       0.85      0.67      0.75      1454

    accuracy                           0.88      5278
   macro avg       0.87      0.81      0.83

### 9. Export your model as a pickle file

In [17]:
# Create a pickle file for the model
with open('model.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.